In [2]:
import pandas as pd
%matplotlib inline

In [3]:
data = pd.read_csv("moviedata.csv")

In [5]:
data.head()

,Month,Total Gross,% Of Year,Avg.,Top Movie of the Month,Gross,% Of Total
0,1/1/2000,591410000,8%,"$3,969,195",Stuart Little,"$51,424,419",9%
1,2/1/2000,511350000,7%,"$3,277,885",Scream 3,"$79,050,098",15%
2,3/1/2000,511830000,7%,"$2,812,253",Erin Brockovich,"$66,216,976",13%
3,4/1/2000,524030000,7%,"$2,994,457",Rules of Engagement,"$50,228,672",10%
4,5/1/2000,637500000,8%,"$3,863,636",Gladiator (2000),"$129,437,265",20%


In [6]:
data.describe()

,Total Gross
count,2.070000e+02
mean,8.171416e+08
std,2.110230e+08
min,4.017100e+08
25%,6.390550e+08
50%,8.067400e+08
75%,9.529880e+08
max,1.398360e+09
